In [1]:
import json
from pathlib import Path

import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("/nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct")
# model = AutoModelForCausalLM.from_pretrained("/nfs/public/hf/models/meta-llama/Meta-Llama-3-8B-Instruct",
#     # token='hf_xxx'
#     device_map="auto",
#     trust_remote_code=True,
#     torch_dtype=torch.bfloat16,
#     local_files_only=True
# )

In [ ]:
TEMPLATES = ['A1']

In [ ]:
for template in TEMPLATES:
    path = Path(f'../resources/data/{template}.json')
    if not path.exists():
        raise FileNotFoundError(f'The template {template} does not exist.')
    with path.open(encoding='utf-8') as f:
        data = json.load(f)

In [ ]:
system_content = "The following text contains a question, followed by a process that was used to solve it. Select the steps in the process that are most relevant for inclusion in a summary of that process. Format your output using a list containing the indices of the selected steps, for example: [1, 2, 4]."

inputs = []
for example in data.values():

    user_content = ""
    user_content += f"Question: {example['question']}. Process: "
    for step in example['explanation']:
        user_content += f"{step['step']}. {step['explanation']}. "

    messages = [
        {
            "role": "system",
            "content": system_content
        },
        {
            "role": "user",
            "content": user_content
        }
    ]

    inputs.append(messages)

In [ ]:
input_messages = [tokenizer.apply_chat_template(i) for i in inputs]
input_ids = [tokenizer.encode(i, return_tensors="pt") for i in inputs_messages]
input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
outputs = model.generate(input_ids)
responses = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

[{'role': 'system',
  'content': 'The following text contains a question, followed by a process that was used to solve it. Select the steps in the process that are most relevant for inclusion in a summary of that process. Format your output using a list containing the indices of the selected steps, for example: [1, 2, 4].'},
 {'role': 'user',
  'content': 'Question: In 2026, what will be the population density of Liechtenstein?. Process: 1. The population density of Liechtenstein in 2026 had to be predicted using historic data. 2. Data was found on the population density of Liechtenstein between 2010 and 2021. 3. A predictive model was created using this data. 4. The answer was estimated using this predictive model. '}]